<a href="https://colab.research.google.com/github/Akash12-sudo/Akash12-sudo.github.io/blob/master/Skin_Disease_Detection_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
dataset_ref = zipfile.ZipFile('/content/drive/MyDrive/dataset.zip')
dataset_ref.extractall('/content')
dataset_ref.close()

In [ ]:
train_dir = '/content/train'
val_dir = '/content/test'

# training dataset generator 
train_datagen = ImageDataGenerator(        
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# validation dataset generator
val_datagen = ImageDataGenerator(
    rescale = 1./255
)

# training dataset
train_ds = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (224, 224),
    class_mode = "categorical",
    batch_size = 32,
)

# validation dataset
val_ds = val_datagen.flow_from_directory(
    val_dir, 
    target_size = (224, 224),
    class_mode = "categorical",
)



Found 6699 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(23, activation = 'softmax')(x)

model = keras.models.Model(inputs = base_model.input, outputs = predictions)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs = 2, validation_data = val_ds, batch_size = 32)

Epoch 1/2
210/210 [==============================] - 171s 809ms/step - loss: 3.5353 - accuracy: 0.1282 - val_loss: 2.8521 - val_accuracy: 0.1777
Epoch 2/2
210/210 [==============================] - 160s 761ms/step - loss: 2.7459 - accuracy: 0.1899 - val_loss: 2.8788 - val_accuracy: 0.1744
